In [56]:
# from sklearn import datasets

# data = datasets.load_iris()
# data.keys()

# def FeatureNormalize(X):
#     mu = X.mean(axis=0)
#     sigma = X.std(axis=0)
#     [m,n]=X.shape
#     t = np.ones((m,n))
    
#     Xnorm = (X-(t*mu))/(t*sigma)
    
#     return(Xnorm)

# def to_one_hot(Y):
#     n_col = np.amax(Y) + 1
#     binarized = np.zeros((len(Y), n_col))
#     for i in range(len(Y)):
#         binarized[i, Y[i]] = 1.
#     return binarized

# #Convert one-hot encoding to array
# def from_one_hot(Y):
#     arr = np.zeros((len(Y), 1))

#     for i in range(len(Y)):
#         l = layer2[i]
#         for j in range(len(l)):
#             if(l[j] == 1):
#                 arr[i] = j+1
#     return arr


# # Feature Linear
# X = np.c_[np.ones((data['data'].shape[0])),data['data']]
# X = FeatureNormalize(X)

# y = data['target'].reshape(-1,1)
# # y = y.flatten()
# # y = to_one_hot(y)

# print('X:',X.shape, '(with intercept)')
# print('y:',y.shape)

In [57]:
# %load ../../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# load MATLAB files
from scipy.io import loadmat

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('darkgrid')

In [58]:
data = loadmat('ex4data1.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [59]:
y = data['y']
# Add intercept
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]

print('X:',X.shape, '(with intercept)')
print('y:',y.shape)

X: (5000, 401) (with intercept)
y: (5000, 1)


In [60]:
# weights = loadmat('ex4weights.mat')
# weights.keys()

In [61]:
# theta1, theta2 = weights['Theta1'], weights['Theta2']
# print('theta1 :', theta1.shape)
# print('theta2 :', theta2.shape)
# params = np.r_[theta1.ravel(), theta2.ravel()]
# print('params :', params.shape)

In [62]:
inputLs=400
hiddenLs=25
numLabels=10

# initial Params

In [63]:
def createRandomParam(Lin,Lout):
    #iParam = np.zeros((Lout,Lin+1))
    eps = 0.12;
    iParam = np.random.rand(Lout,Lin+1)* 2 * eps - eps;

    return(iParam)

In [64]:
iTheta1 = createRandomParam(inputLs,hiddenLs)
iTheta2 = createRandomParam(hiddenLs,numLabels)
initialParams= np.r_[iTheta1.ravel(),iTheta2.ravel()]
print('iTheta1',iTheta1.shape)
print('iTheta2',iTheta2.shape)
print('initialParams',initialParams.shape)
type(iTheta1)

iTheta1 (25, 401)
iTheta2 (10, 26)
initialParams (10285,)


numpy.ndarray

<b>Neural Network</b> <br>
Input layer size = 400 (20x20 pixels)  <br>
Hidden layer size = 25 <br>
Number of labels = 10 <br>

# Neural Networks - Feed Forward and Cost Function

In [65]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

In [66]:
def sigmoidGradient(z):
    return(sigmoid(z)*(1-sigmoid(z)))

In [67]:
def getOptTheta(nn_params, input_layer_size, hidden_layer_size, num_labels, features, classes, reg,alpha,num_iter):
    
    # When comparing to Octave code note that Python uses zero-indexed arrays.
    # But because Numpy indexing does not include the right side, the code is the same anyway.
    theta1 = nn_params[0:(hidden_layer_size*(input_layer_size+1))].reshape(hidden_layer_size,(input_layer_size+1))
    theta2 = nn_params[(hidden_layer_size*(input_layer_size+1)):].reshape(num_labels,(hidden_layer_size+1))

    m = features.shape[0]
    y_matrix = pd.get_dummies(classes.ravel()).as_matrix() 
    
    for i in np.arange(num_iter):
        # Cost
        a1 = features # 5000x401

        z2 = theta1.dot(a1.T) # 25x401 * 401x5000 = 25x5000 
        a2 = np.c_[np.ones((features.shape[0],1)),sigmoid(z2.T)] # 5000x26 

        z3 = theta2.dot(a2.T) # 10x26 * 26x5000 = 10x5000 
        a3 = sigmoid(z3) # 10x5000

        #J = -1*(1/m)*np.sum((np.log(a3.T)*(y_matrix) + np.log(1-a3).T*(1-y_matrix))) + \
        #    (reg/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))

        # Gradients
        d3 = a3.T - y_matrix # 5000x10
        d2 = theta2[:,1:].T.dot(d3.T)*sigmoidGradient(z2) # 25x10 *10x5000 * 25x5000 = 25x5000

        delta1 = d2.dot(a1) # 25x5000 * 5000x401 = 25x401
        delta2 = d3.T.dot(a2) # 10x5000 *5000x26 = 10x26

        theta1_ = np.c_[np.ones((theta1.shape[0],1)),theta1[:,1:]]
        theta2_ = np.c_[np.ones((theta2.shape[0],1)),theta2[:,1:]]

        theta1_grad = delta1/m + (theta1_*reg)/m
        theta2_grad = delta2/m + (theta2_*reg)/m

        theta1 = theta1 - (alpha*theta1_grad)
        theta2 = theta2 - (alpha*theta2_grad)
        
    return(theta1,theta2)

In [ ]:
# inputLs=400
# hiddenLs=25
# numLabels=10
# inputLs=4
# hiddenLs=25
# numLabels=3
#-------
reg=0
alpha=0.1
numIter=15000

tOpt1,tOpt2 = getOptTheta(initialParams,inputLs,hiddenLs,numLabels,X,y,reg,alpha,numIter)

In [ ]:
# initialThetaTxt = np.loadtxt('iTheta.txt')
# input_layer_size=400
# hidden_layer_size=25
# num_labels=10

# tOpt1 = initialThetaTxt[0:(hidden_layer_size*(input_layer_size+1))].reshape(hidden_layer_size,(input_layer_size+1))
# tOpt2 = initialThetaTxt[(hidden_layer_size*(input_layer_size+1)):].reshape(num_labels,(hidden_layer_size+1))

In [ ]:
tOpt1.shape

In [ ]:
tOpt1

In [ ]:
tOpt2.shape

In [ ]:
tOpt2

# Prediction

In [ ]:
def prediction(X,thetaOpt1,thetaOpt2):
    [m,n] = X.shape
    #p = np.zeros((m,1))
    
    #thetaOpt1 = 25x401
    #thetaOpt2 = 10x26
    
    a1 = X #5000x401
    a2 = sigmoid(X.dot(thetaOpt1.T) ) #5000x25
    hypo = sigmoid(np.c_[np.ones((a2.shape[0],1)),a2].dot(thetaOpt2.T)) #5000x10
    
    opt = (np.argmax(hypo, axis=1))+1
    opt = opt.reshape(-1,1)
    return(opt)  

In [ ]:
#OUR FUNCTION
# pred = prediction(X,thetaOpt1,thetaOpt2)

# THETA OPTIMUM OCTAVE
pred = prediction(X,tOpt1,tOpt2)
accuracy = np.mean(pred==y)*100
print('accuracy NN={}%'.format(accuracy))